In [37]:
import os

In [38]:
%pwd

'c:\\Users\\Amogh'

In [39]:
os.chdir("../")

In [40]:
%pwd

'c:\\Users'

In [41]:
from dataclasses import dataclass
from pathlib import Path
#entity
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL:str
    local_data_file:Path
    unzip_dir:Path


In [42]:
from BriefChat.constants import *
from BriefChat.utils .common import read_yaml,create_directories

In [43]:
class ConfigurationManager:
    def __init__(
            self,
            config_file_path=CONFIG_FILE_PATH,
            params_file_path=PARAMS_FILE_PATH):
        self.config=read_yaml(config_file_path)
        self.params=read_yaml(params_file_path)
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config
        
    

In [44]:
import os
import urllib.request as request
import zipfile
from BriefChat.logging import logger
from BriefChat.utils.common import get_size




In [45]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  

        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [46]:
os.chdir(r'C:\Users\Amogh\Desktop\BriefChat')

try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-01-22 00:17:59,435: INFO:common: yaml file: config\config.yaml loaded successfully]
[2024-01-22 00:17:59,462: INFO:common: yaml file: params.yaml loaded successfully]
[2024-01-22 00:17:59,467: INFO:common: created directory at: artifacts]
[2024-01-22 00:17:59,469: INFO:common: created directory at: artifacts/data_ingestion]


[2024-01-22 00:18:02,427: INFO:1434958058: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 4187821
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "180e88c0674d375f5a74d75d4757055eff4e0be8bdb2b2319af7e8b2a12e1543"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: FACE:3CF775:98D59:F2B73:65AD66E2
Accept-Ranges: bytes
Date: Sun, 21 Jan 2024 18:48:03 GMT
Via: 1.1 varnish
X-Served-By: cache-bom4735-BOM
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1705862883.929439,VS0,VE508
Vary: Authorization,Accept-Encoding,Origin
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-Fastly-Request-ID: 0225499c205432fec97ee8ff92578259932a10ef
Expires: Sun, 21 Jan 2024 18:53:03 GMT
Source-Age: 0

]
